# Punto 3 Comparación de Rendimiento con GLPK/Pyomo

## Solución por medio de Simplex

Gilmoire Girls

In [1]:
import numpy as np
import pandas as pd

Para el primer paso se ingresan los datos del enunciado, incluyendo la matriz de coeficientes de restricciones (A), los terminos independientes y los coeficientes d ela función objetivo.

In [2]:

restricciones = np.array([
    [1, 2, 1, 1, 0, 0, 3, 1, 2, 1], 
    [2, 1, 0, 2, 1, 1, 0, 3, 1, 2],
    [1, 1, 2, 0, 2, 1, 1, 0, 3, 1],
    [0, 2, 1, 1, 1, 0, 2, 1, 1, 1],
    [2, 0, 1, 1, 1, 2, 1, 1, 0, 2],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 2, 1, 0, 1, 2, 1, 1, 0],
    [1, 0, 1, 2, 1, 0, 1, 2, 1, 1]
], dtype=float)

soluciones = np.array([50, 60, 55, 40, 45, 70, 65, 50], dtype=float).reshape(-1, 1)
z = np.array([[-5, -8, -3, -7, -6, -9, -4, -10, -2, -11]], dtype=float)


Como todas las restricciones son de la forma $\le$, se tienen que crear las variables de holgura para cada una de las filas para que las restricciones sean de tipo $=$:


In [3]:
num_restricciones = restricciones.shape[0]
holguras = np.eye(num_restricciones)
restricciones_conholguras = np.hstack((restricciones, holguras))
#Toca añadir un array de 0s a la fila de Z
z_con_holguras = np.hstack((z, np.zeros((1, num_restricciones))))

In [4]:
tabla = np.hstack((restricciones_conholguras, soluciones))
tabla = np.vstack((tabla, np.append(z_con_holguras, 0)))

columnas = [f'x{i+1}' for i in range(10)]
holguras = [f'h{i+1}' for i in range(num_restricciones)]
real_col = columnas + holguras + ["solucion"]
index = holguras + ["Z"]

df = pd.DataFrame(tabla, columns=real_col, index=index)
print(df)

     x1   x2   x3   x4   x5   x6   x7    x8   x9   x10   h1   h2   h3   h4  \
h1  1.0  2.0  1.0  1.0  0.0  0.0  3.0   1.0  2.0   1.0  1.0  0.0  0.0  0.0   
h2  2.0  1.0  0.0  2.0  1.0  1.0  0.0   3.0  1.0   2.0  0.0  1.0  0.0  0.0   
h3  1.0  1.0  2.0  0.0  2.0  1.0  1.0   0.0  3.0   1.0  0.0  0.0  1.0  0.0   
h4  0.0  2.0  1.0  1.0  1.0  0.0  2.0   1.0  1.0   1.0  0.0  0.0  0.0  1.0   
h5  2.0  0.0  1.0  1.0  1.0  2.0  1.0   1.0  0.0   2.0  0.0  0.0  0.0  0.0   
h6  1.0  1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0  0.0  0.0  0.0  0.0   
h7  0.0  1.0  2.0  1.0  0.0  1.0  2.0   1.0  1.0   0.0  0.0  0.0  0.0  0.0   
h8  1.0  0.0  1.0  2.0  1.0  0.0  1.0   2.0  1.0   1.0  0.0  0.0  0.0  0.0   
Z  -5.0 -8.0 -3.0 -7.0 -6.0 -9.0 -4.0 -10.0 -2.0 -11.0  0.0  0.0  0.0  0.0   

     h5   h6   h7   h8  solucion  
h1  0.0  0.0  0.0  0.0      50.0  
h2  0.0  0.0  0.0  0.0      60.0  
h3  0.0  0.0  0.0  0.0      55.0  
h4  0.0  0.0  0.0  0.0      40.0  
h5  1.0  0.0  0.0  0.0      45.0  
h6  0.0  

### Definifición de funciones del punto 2

Para realizar la optimización se tienen 3 funciones principales:
- revisar_maximizacion(tabla): revisa la condición de parada de la tabla de simplex. Si encuentra que hay algún coeficiente negativo en la fila de $Z$, significa que aún se puede optimizar más. Si no encuentra ningún negativo, ya se llegó al óptimo. 
- pivote(tabla, num_fila_piv, num_col_piv): Reacibiendo la tabla actual, el numero de la columna que entra y la fila que sale, realiza las transformaciones necesarias para realizar el pivote.
- solver_minimizacion(tabla): Lógica principal del proceso de simplex. Recibe la tabla con los coeficientes de ambos las variables de restricción y las variables de holgura, y realiza el proceso de optimización por medio de simplex hasta que revisar_maximizacion devuelva False.

In [5]:
def revisar_maximizacion(tabla):
    return any(var<0 for var in tabla[-1,:-1])

def pivote(tabla, num_fila_piv, num_col_piv):
    nueva_tabla = tabla.copy()

    #Se toma solo la fila pivote, se edita y se devuelve a la tabla
    fila_pivote = nueva_tabla[num_fila_piv, :].copy()
    pivote_elemento = fila_pivote[num_col_piv]
    fila_pivote = fila_pivote / pivote_elemento
    nueva_tabla[num_fila_piv, :] = fila_pivote

    for i in range(nueva_tabla.shape[0]):
        if i != num_fila_piv:
            fila_actual = nueva_tabla[i, :].copy()
            coef = fila_actual[num_col_piv]
            nueva_tabla[i, :] = fila_actual - (coef * fila_pivote)

    return nueva_tabla




def solver_minimizacion(tabla, filas, columnas):
    iteracion = 1
    
    while revisar_maximizacion(tabla):
        print (f"Iteración {iteracion}:")
        columna = np.argmin(tabla[-1,:-1])
        print (f"Columna pivote:  {columna}. Entra la variable {columnas[columna]} como básica")       
        #Encontrar la fila pivote
        num_fila = 0
        ratios = []
        for i in range(tabla[:-1].shape[0]):
            if tabla[i, columna] > 0:
                posible = tabla[i, -1] / tabla[i, columna]
                ratios.append(posible)
            else:
                ratios.append(np.inf)

        
        num_fila = np.argmin(ratios)
        print (f"Fila pivote:  {num_fila}. Sale la variable {filas[num_fila]} como básica.") 
        filas[num_fila] = columnas[columna]
        tabla = pivote(tabla, num_fila, columna)
        print("tabla actual:")
        print (pd.DataFrame(tabla, columns=columnas, index=filas))
        iteracion+=1
    print("Solución encontrada!!: ")
    return tabla


        

Da una solución viable. Posteriormente se generará una función que agrupe todos los procesos necesarios para resolver el proble,a y poder hacer una toma de tiempo

# Automatización del proceso para medir tiempos


In [6]:
import time

Esta es la función que recibe los datos de entrada (`restricciones`, `soluciones`, `z`) y calcula el óptimo, primero realizando las transformaciones necesarias a la tabla original para crear las variables de holgura, y luego realizando el llamado a `solver_minimización`

In [7]:
def automatizacion(restricciones,soluciones,z):
    num_restricciones = restricciones.shape[0]
    holguras = np.eye(num_restricciones)
    restricciones_conholguras = np.hstack((restricciones, holguras))
    #Toca añadir un array de 0s a la fila de Z
    z_con_holguras = np.hstack((z, np.zeros((1, num_restricciones))))
    tabla = np.hstack((restricciones_conholguras, soluciones))
    tabla = np.vstack((tabla, np.append(z_con_holguras, 0)))

    columnas = [f'x{i+1}' for i in range(10)]
    holguras = [f'h{i+1}' for i in range(num_restricciones)]
    real_col = columnas + holguras + ["solucion"]
    index = holguras + ["Z"]
    tabla = solver_minimizacion(tabla, index, real_col)
    df = pd.DataFrame(tabla, columns=real_col, index=index)
    print(df)

start_time = time.time()
automatizacion(restricciones=restricciones, soluciones= soluciones, z=z)
entd_time = time.time()
print(f"El tiempo que se demoró en encontrar el óptimo fue de {entd_time-start_time}ms")

Iteración 1:
Columna pivote:  9. Entra la variable x10 como básica
Fila pivote:  4. Sale la variable h5 como básica.
tabla actual:
      x1   x2   x3   x4   x5   x6   x7   x8   x9  x10   h1   h2   h3   h4  \
h1   0.0  2.0  0.5  0.5 -0.5 -1.0  2.5  0.5  2.0  0.0  1.0  0.0  0.0  0.0   
h2   0.0  1.0 -1.0  1.0  0.0 -1.0 -1.0  2.0  1.0  0.0  0.0  1.0  0.0  0.0   
h3   0.0  1.0  1.5 -0.5  1.5  0.0  0.5 -0.5  3.0  0.0  0.0  0.0  1.0  0.0   
h4  -1.0  2.0  0.5  0.5  0.5 -1.0  1.5  0.5  1.0  0.0  0.0  0.0  0.0  1.0   
x10  1.0  0.0  0.5  0.5  0.5  1.0  0.5  0.5  0.0  1.0  0.0  0.0  0.0  0.0   
h6   0.0  1.0  0.5  0.5  0.5  0.0  0.5  0.5  1.0  0.0  0.0  0.0  0.0  0.0   
h7   0.0  1.0  2.0  1.0  0.0  1.0  2.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
h8   0.0  0.0  0.5  1.5  0.5 -1.0  0.5  1.5  1.0  0.0  0.0  0.0  0.0  0.0   
Z    6.0 -8.0  2.5 -1.5 -0.5  2.0  1.5 -4.5 -2.0  0.0  0.0  0.0  0.0  0.0   

      h5   h6   h7   h8  solucion  
h1  -0.5  0.0  0.0  0.0      27.5  
h2  -1.0  0.0  0.0  0.0   

## Solución con pyomo

In [ ]:
from pyomo.environ import *

model = ConcreteModel()
model.T= RangeSet(1,10)
c = [5, 8, 3, 7, 6, 9, 4, 10, 2, 11]
model.x = Var(model.T, domain=NonNegativeReals)
model.obj = Objective(expr=sum(c[i-1]*model.x[i] for i in model.T), sense=maximize)
model.tareas = ConstraintList()


A = [
    [1, 2, 1, 1, 0, 0, 3, 1, 2, 1],
    [2, 1, 0, 2, 1, 1, 0, 3, 1, 2],
    [1, 1, 2, 0, 2, 1, 1, 0, 3, 1],
    [0, 2, 1, 1, 1, 0, 2, 1, 1, 1],
    [2, 0, 1, 1, 1, 2, 1, 1, 0, 2],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 2, 1, 0, 1, 2, 1, 1, 0],
    [1, 0, 1, 2, 1, 0, 1, 2, 1, 1]
]
b = [50, 60, 55, 40, 45, 70, 65, 50]
for j in range(8):
    model.tareas.add(expr=sum(A[j][i] * model.x[i + 1] for i in range(10)) <= b[j])

for n in range(10):
    model.tareas.add(model.x[n+1] >=0)

from pyomo.opt import SolverFactory
start_time_2 = time.time()
solver = SolverFactory('glpk')
#solver.options['display'] = 'iter'
solver.solve(model, )
entd_time_time_2 = time.time()

print("Máximo valor:", model.obj())
print("Coeficientes a cada variable:")
for i in model.T:
    print(f"Variable x_{i}: {model.x[i].value}")
print(f"El tiempo que se demoró en encontrar el óptimo fue de {entd_time_time_2-start_time_2}ms")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --display iter --write C:\Users\57304\AppData\Local\Temp\tmpoo42czgu.glpk.raw
 --wglp C:\Users\57304\AppData\Local\Temp\tmpmuxqeil1.glpk.glp --cpxlp C:\Users\57304\AppData\Local\Temp\tmpvdacsalw.pyomo.lp
Reading problem data from 'C:\Users\57304\AppData\Local\Temp\tmpvdacsalw.pyomo.lp'...
18 rows, 10 columns, 75 non-zeros
158 lines were read
Writing problem data to 'C:\Users\57304\AppData\Local\Temp\tmpmuxqeil1.glpk.glp'...
134 lines were written
GLPK Simplex Optimizer 5.0
18 rows, 10 columns, 75 non-zeros
Preprocessing...
8 rows, 10 columns, 65 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.000e+00  ratio =  3.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (10)
*     4: obj =   3.756250000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (43463 b